In [1]:
!rmdir /S /Q lightning_logs

In [2]:
import feature_engine.encoding
import feature_engine.pipeline
import feature_engine.selection
import feature_engine.wrappers
import lightning
import pandas
import sklearn.preprocessing
import sweetviz
import torch
import torchmetrics

In [3]:
lightning.seed_everything(0)

Seed set to 0


0

In [4]:
train_df = pandas.read_csv("train.csv")
train_df

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,20753,Male,25.137087,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,2.151809,no,1.330519,0.196680,Sometimes,Public_Transportation,Obesity_Type_II
20754,20754,Male,18.000000,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,1.000000,no,2.000000,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
20755,20755,Male,20.101026,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,2.000000,no,1.158040,1.198439,no,Public_Transportation,Obesity_Type_II
20756,20756,Male,33.852953,1.700000,83.520113,yes,yes,2.671238,1.971472,Sometimes,no,2.144838,no,0.000000,0.973834,no,Automobile,Overweight_Level_II


In [5]:
# report = sweetviz.analyze(train_df)
# report.show_notebook()

In [6]:
train_df.isnull().sum()

id                                0
Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [7]:
train_X_df = train_df.drop(columns=["NObeyesdad"])
train_X_df

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,20753,Male,25.137087,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,2.151809,no,1.330519,0.196680,Sometimes,Public_Transportation
20754,20754,Male,18.000000,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,1.000000,no,2.000000,1.000000,Sometimes,Public_Transportation
20755,20755,Male,20.101026,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,2.000000,no,1.158040,1.198439,no,Public_Transportation
20756,20756,Male,33.852953,1.700000,83.520113,yes,yes,2.671238,1.971472,Sometimes,no,2.144838,no,0.000000,0.973834,no,Automobile


In [8]:
train_y_df = train_df[["NObeyesdad"]]
train_y_df

,NObeyesdad
0,Overweight_Level_II
1,Normal_Weight
2,Insufficient_Weight
3,Obesity_Type_III
4,Overweight_Level_II
...,...
20753,Obesity_Type_II
20754,Insufficient_Weight
20755,Obesity_Type_II
20756,Overweight_Level_II


In [9]:
pipeline = feature_engine.pipeline.make_pipeline(
    feature_engine.selection.DropFeatures(features_to_drop=["id"]),   
    feature_engine.wrappers.SklearnTransformerWrapper(transformer=sklearn.preprocessing.MinMaxScaler(), variables=["Age", "Height", "Weight", "FCVC", "NCP", "CH2O", "FAF", "TUE"]),
    feature_engine.encoding.OrdinalEncoder(encoding_method='arbitrary', variables=["Gender", "family_history_with_overweight", "FAVC", "CAEC", "SMOKE", "SCC", "CALC", "MTRANS"]),
)

In [10]:
train_X_transformed_df = pipeline.fit_transform(train_X_df)
train_X_transformed_df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,0,0.222192,0.475586,0.338497,0,0,0.500000,0.661099,0,0,0.881787,0,0.000000,0.488237,0,0
1,1,0.085106,0.209260,0.142792,0,0,0.500000,0.666667,1,0,0.500000,0,0.333333,0.500000,1,1
2,1,0.085106,0.497391,0.088577,0,0,0.440267,0.137228,0,0,0.455189,0,0.288682,0.836792,1,0
3,1,0.147931,0.496002,0.732007,0,0,1.000000,0.666667,0,0,0.337031,0,0.489288,0.390099,0,0
4,0,0.375342,0.883049,0.434708,0,0,0.839832,0.323824,0,0,0.489924,0,0.655991,0.465861,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,0,0.236959,0.602336,0.596452,0,0,0.959792,0.666667,0,0,0.575905,0,0.443506,0.098340,0,0
20754,0,0.085106,0.494613,0.087262,1,0,1.000000,1.000000,1,0,0.000000,0,0.666667,0.500000,0,0
20755,0,0.129809,0.703030,0.528177,0,0,0.703909,0.666667,0,0,0.500000,0,0.386013,0.599220,1,0
20756,0,0.422403,0.475590,0.353174,0,0,0.835619,0.323824,0,0,0.572419,0,0.000000,0.486917,1,1


In [11]:
target_encoder = feature_engine.encoding.OrdinalEncoder(encoding_method="arbitrary", variables=["NObeyesdad"])
train_y_transformed_df = target_encoder.fit_transform(train_y_df)
train_y_transformed_df

,NObeyesdad
0,0
1,1
2,2
3,3
4,0
...,...
20753,4
20754,2
20755,4
20756,0


In [12]:
train_transformed_df = train_X_transformed_df.join(train_y_transformed_df)
train_transformed_df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,0.222192,0.475586,0.338497,0,0,0.500000,0.661099,0,0,0.881787,0,0.000000,0.488237,0,0,0
1,1,0.085106,0.209260,0.142792,0,0,0.500000,0.666667,1,0,0.500000,0,0.333333,0.500000,1,1,1
2,1,0.085106,0.497391,0.088577,0,0,0.440267,0.137228,0,0,0.455189,0,0.288682,0.836792,1,0,2
3,1,0.147931,0.496002,0.732007,0,0,1.000000,0.666667,0,0,0.337031,0,0.489288,0.390099,0,0,3
4,0,0.375342,0.883049,0.434708,0,0,0.839832,0.323824,0,0,0.489924,0,0.655991,0.465861,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,0,0.236959,0.602336,0.596452,0,0,0.959792,0.666667,0,0,0.575905,0,0.443506,0.098340,0,0,4
20754,0,0.085106,0.494613,0.087262,1,0,1.000000,1.000000,1,0,0.000000,0,0.666667,0.500000,0,0,2
20755,0,0.129809,0.703030,0.528177,0,0,0.703909,0.666667,0,0,0.500000,0,0.386013,0.599220,1,0,4
20756,0,0.422403,0.475590,0.353174,0,0,0.835619,0.323824,0,0,0.572419,0,0.000000,0.486917,1,1,0


In [13]:
class MyModel(lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self._stack = torch.nn.Sequential(
            torch.nn.Linear(16, 32), torch.nn.ReLU(),
            torch.nn.Linear(32, 7), torch.nn.Softmax(dim=1),
        )
        self._accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=7)

    def forward(self, x):
        return self._stack(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def training_step(self, batch, batch_idx):
        (loss, acc) = self._compute(batch, batch_idx)
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def validation_step(self, batch, batch_idx):
        (loss, acc) = self._compute(batch, batch_idx)
        self.log("val_loss", loss)
        self.log("val_acc", acc)
        return loss

    def test_step(self, batch, batch_idx):
        (loss, acc) = self._compute(batch, batch_idx)
        self.log("test_loss", loss)
        self.log("test_acc", acc)
        return loss
    
    def _compute(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        pred = y_hat.argmax(axis=1)
        loss = torch.nn.functional.cross_entropy(y_hat, y)
        acc = self._accuracy(pred, y)
        return (loss, acc)

In [14]:
class MyDataModule(lightning.LightningDataModule):
    def __init__(self):
        super().__init__()
        train_X_tensor = torch.tensor(train_X_transformed_df.values, dtype=torch.float32)
        train_y_tensor = torch.tensor(train_y_transformed_df["NObeyesdad"].values, dtype=torch.int64)
        full_dataset = torch.utils.data.TensorDataset(train_X_tensor, train_y_tensor)
        n_train = int(len(full_dataset) * 0.8)
        n_val = int(len(full_dataset) * 0.1)
        n_test = len(full_dataset) - n_train - n_val
        self._train_dataset, self._val_dataset, self._test_dataset = torch.utils.data.random_split(full_dataset, [n_train, n_val, n_test])

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self._train_dataset, batch_size=512)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self._val_dataset, batch_size=512)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self._test_dataset)

In [15]:
trainer = lightning.Trainer(max_epochs=500, callbacks=[lightning.pytorch.callbacks.early_stopping.EarlyStopping(monitor="val_loss", mode="min")])
model = MyModel()
loader = MyDataModule()
trainer.fit(model, datamodule=loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | _stack    | Sequential         | 775    | train
1 | _accuracy | MulticlassAccuracy | 0      | train
---------------------------------------------------------
775       Trainable params
0         Non-trainable params
775       Total params
0.003     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                               | 0/? [00:00<…

C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (33) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=500` reached.


In [17]:
!taskkill /im tensorboard.exe /f
!del /q %TMP%\.tensorboard-info\*
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

SUCCESS: The process "tensorboard.exe" with PID 20552 has been terminated.


In [18]:
trainer.test(datamodule=loader)

C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at C:\Users\hsmtk\repo\fantastic-octo-dollop\obesity\lightning_logs\version_0\checkpoints\epoch=499-step=16500.ckpt
Loaded model weights from the checkpoint at C:\Users\hsmtk\repo\fantastic-octo-dollop\obesity\lightning_logs\version_0\checkpoints\epoch=499-step=16500.ckpt
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1

Testing: |                                                                                       | 0/? [00:00<…

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8757823705673218
        test_loss            1.293918251991272
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.293918251991272, 'test_acc': 0.8757823705673218}]